In [1]:
from pathlib import Path
import numpy as np
import rasterio
from rasterio.warp import reproject, Resampling
from rasterio.transform import Affine

# ----------------------------
# A) Paths
# ----------------------------
project_root = Path.cwd().resolve()
# If your notebook runs from /notebooks, move up one level automatically:
if project_root.name.lower() == "notebooks":
    project_root = project_root.parent

raw_dir = project_root / "output" / "raw"
out_dir = project_root / "output" / "processed"
out_dir.mkdir(parents=True, exist_ok=True)

s2_t1 = raw_dir / "s2_t1.tif"
s2_t2 = raw_dir / "s2_t2.tif"
s1_t1 = raw_dir / "s1_t1.tif"
s1_t2 = raw_dir / "s1_t2.tif"

for p in [s2_t1, s2_t2, s1_t1, s1_t2]:
    if not p.exists():
        raise FileNotFoundError(f"Missing file: {p}")

print("Raw files found ✅")
print(" -", s2_t1)
print(" -", s2_t2)
print(" -", s1_t1)
print(" -", s1_t2)

# ----------------------------
# B) Helper: choose UTM EPSG from ROI centroid (better for 10m grids)
# ----------------------------
def utm_epsg_from_lonlat(lon: float, lat: float) -> int:
    zone = int((lon + 180) // 6) + 1
    return (32600 + zone) if lat >= 0 else (32700 + zone)

# We'll use s2_t1 bounds to estimate centroid lon/lat
with rasterio.open(s2_t1) as src:
    # if raster is EPSG:4326, bounds are lon/lat; if not, still ok for rough centroid
    b = src.bounds
    cx = (b.left + b.right) / 2
    cy = (b.bottom + b.top) / 2
    src_crs = src.crs

# If src is not EPSG:4326, centroid isn't lon/lat; but your pipeline was in EPSG:4326 bbox,
# so typically it is EPSG:4326.
target_epsg = utm_epsg_from_lonlat(cx, cy)
print("Target UTM EPSG:", target_epsg)

# ----------------------------
# C) Coregistration strategy (what we will do)
# 1) Reproject each raster to SAME CRS (UTM)
# 2) Snap all rasters to ONE reference grid (same transform, width/height)
#    Reference = reprojected s2_t1
# ----------------------------

def reproject_to_target_grid(
    src_path: Path,
    dst_path: Path,
    dst_crs,
    ref_transform: Affine,
    ref_width: int,
    ref_height: int,
    resampling_map,
    force_nodata: float = -9999.0,
):
    with rasterio.open(src_path) as src:
        src_meta = src.meta.copy()
        src_nodata = src.nodata
        if src_nodata is None:
            src_nodata = force_nodata

        count = src.count
        dst_meta = src_meta.copy()
        dst_meta.update({
            "crs": dst_crs,
            "transform": ref_transform,
            "width": ref_width,
            "height": ref_height,
            "nodata": src_nodata,
            "compress": "deflate"
        })

        # create empty destination array
        dst_data = np.full((count, ref_height, ref_width), src_nodata, dtype=src.dtypes[0])

        with rasterio.open(dst_path, "w", **dst_meta) as dst:
            for b in range(1, count + 1):
                # choose resampling per band
                rs = resampling_map(b, count)

                reproject(
                    source=rasterio.band(src, b),
                    destination=dst_data[b - 1],
                    src_transform=src.transform,
                    src_crs=src.crs,
                    src_nodata=src_nodata,
                    dst_transform=ref_transform,
                    dst_crs=dst_crs,
                    dst_nodata=src_nodata,
                    resampling=rs,
                )
                dst.write(dst_data[b - 1], b)

    print("Saved:", dst_path.name)

# ----------------------------
# D) Build the REFERENCE GRID from s2_t1 at 10m in UTM
# ----------------------------
from rasterio.warp import calculate_default_transform

with rasterio.open(s2_t1) as src:
    # create a ~10m UTM grid from s2_t1
    dst_crs = rasterio.crs.CRS.from_epsg(target_epsg)

    # IMPORTANT: request 10m output grid
    # calculate_default_transform chooses resolution unless we pass it:
    transform, width, height = calculate_default_transform(
        src.crs, dst_crs, src.width, src.height, *src.bounds, resolution=10
    )

# We'll use this as the SNAP grid for everything
ref_transform, ref_width, ref_height = transform, width, height
print("Reference grid:")
print(" - CRS:", dst_crs)
print(" - size:", ref_width, "x", ref_height)
print(" - pixel:", (ref_transform.a, -ref_transform.e), "meters")

# ----------------------------
# E) Resampling rules (important!)
# - Continuous data (reflectance/backscatter): bilinear
# - Categorical masks (SCL, cloudMask): nearest
#   Your S2 has 4 bands: B03, B08, SCL, cloudMask
# ----------------------------
def s2_resampling_map(band_index: int, band_count: int):
    # Band 3=SCL (class), Band 4=cloudMask (0/1) => nearest
    if band_index in (3, 4):
        return Resampling.nearest
    return Resampling.bilinear

def s1_resampling_map(band_index: int, band_count: int):
    # VV/VH continuous => bilinear
    return Resampling.bilinear

# ----------------------------
# F) Reproject + SNAP ALL to the SAME grid
# ----------------------------
out_s2_t1 = out_dir / "s2_t1_coreg.tif"
out_s2_t2 = out_dir / "s2_t2_coreg.tif"
out_s1_t1 = out_dir / "s1_t1_coreg.tif"
out_s1_t2 = out_dir / "s1_t2_coreg.tif"

reproject_to_target_grid(s2_t1, out_s2_t1, dst_crs, ref_transform, ref_width, ref_height, s2_resampling_map)
reproject_to_target_grid(s2_t2, out_s2_t2, dst_crs, ref_transform, ref_width, ref_height, s2_resampling_map)
reproject_to_target_grid(s1_t1, out_s1_t1, dst_crs, ref_transform, ref_width, ref_height, s1_resampling_map)
reproject_to_target_grid(s1_t2, out_s1_t2, dst_crs, ref_transform, ref_width, ref_height, s1_resampling_map)

print("\nSTEP 4 DONE ✅ Coregistered files created in:")
print(out_dir)
for p in [out_s2_t1, out_s2_t2, out_s1_t1, out_s1_t2]:
    print(" -", p.name)


Raw files found ✅
 - D:\IIT DHANBAD M.TECH\Image_analysis\output\raw\s2_t1.tif
 - D:\IIT DHANBAD M.TECH\Image_analysis\output\raw\s2_t2.tif
 - D:\IIT DHANBAD M.TECH\Image_analysis\output\raw\s1_t1.tif
 - D:\IIT DHANBAD M.TECH\Image_analysis\output\raw\s1_t2.tif
Target UTM EPSG: 32643
Reference grid:
 - CRS: EPSG:32643
 - size: 6756 x 6892
 - pixel: (10.0, 10.0) meters
Saved: s2_t1_coreg.tif
Saved: s2_t2_coreg.tif
Saved: s1_t1_coreg.tif
Saved: s1_t2_coreg.tif

STEP 4 DONE ✅ Coregistered files created in:
D:\IIT DHANBAD M.TECH\Image_analysis\output\processed
 - s2_t1_coreg.tif
 - s2_t2_coreg.tif
 - s1_t1_coreg.tif
 - s1_t2_coreg.tif


In [2]:
import rasterio
from pathlib import Path

project_root = Path.cwd().resolve()
if project_root.name.lower() == "notebooks":
    project_root = project_root.parent

pdir = project_root / "output" / "processed"
paths = [
    pdir / "s2_t1_coreg.tif",
    pdir / "s2_t2_coreg.tif",
    pdir / "s1_t1_coreg.tif",
    pdir / "s1_t2_coreg.tif",
]

for p in paths:
    with rasterio.open(p) as src:
        print("\n", p.name)
        print(" CRS:", src.crs)
        print(" size:", src.width, "x", src.height, "| bands:", src.count)
        print(" transform:", src.transform)
        print(" nodata:", src.nodata)



 s2_t1_coreg.tif
 CRS: EPSG:32643
 size: 6756 x 6892 | bands: 4
 transform: | 10.00, 0.00, 489440.26|
| 0.00,-10.00, 3564303.20|
| 0.00, 0.00, 1.00|
 nodata: -9999.0

 s2_t2_coreg.tif
 CRS: EPSG:32643
 size: 6756 x 6892 | bands: 4
 transform: | 10.00, 0.00, 489440.26|
| 0.00,-10.00, 3564303.20|
| 0.00, 0.00, 1.00|
 nodata: -9999.0

 s1_t1_coreg.tif
 CRS: EPSG:32643
 size: 6756 x 6892 | bands: 2
 transform: | 10.00, 0.00, 489440.26|
| 0.00,-10.00, 3564303.20|
| 0.00, 0.00, 1.00|
 nodata: -9999.0

 s1_t2_coreg.tif
 CRS: EPSG:32643
 size: 6756 x 6892 | bands: 2
 transform: | 10.00, 0.00, 489440.26|
| 0.00,-10.00, 3564303.20|
| 0.00, 0.00, 1.00|
 nodata: -9999.0
